# Prep 1: steps to install the packages

In [0]:

!pip install py_entitymatching
!pip install scipy
!pip install numpy
!pip install pandas

In [0]:
#from google.colab import files
#uploaded = files.upload()

# [NEED MODIFICATION: Modify this cell to point to the file location]
# Prep 2: enter the file location on your harddisk

In [0]:
table_a = 'IMDB.csv'
table_b = 'Metacritic.csv'
candidate_set = 'Job_Movie_apply_rules_ds.csv'
prediction_set = 'apply_classifier_1.csv'

# Prep 3: reading the files into pandas dataframe

In [0]:
import pandas as pd

dfa = pd.read_csv(table_a)
dfb = pd.read_csv(table_b)
dfc = pd.read_csv(candidate_set)
dfp = pd.read_csv(prediction_set)

# Module: debug_blocker
# Description: debug the blocking rule using the below script to ensure you are not dropping true matches
# Note: You need to run Prep 1 and 2 in order to run this

In [0]:
# Example input format:
# Format of table_a:
# _id, attribute1, attribute2, ....., attributen

# Format of table_b:
# _id, attribute1, attribute2, ....., attributen

# Format of candidate_set
# A_id,B_id
# where A_id is _id from table_a and B_id is the _id column value from table_b

In [0]:
import py_entitymatching as em

def run_debug_blocker(table_a, table_b, table_a_key, table_b_key, candidate_set):
    dfl = em.read_csv_metadata(table_a, key=table_a_key)
    dfr = em.read_csv_metadata(table_b, key=table_b_key)

    # reading the candidate set and adding key
    dfcand = pd.read_csv(candidate_set)
    dfcand.drop_duplicates(inplace=True)
    dfcand.to_csv('cand_set_with_index.csv', index_label='id')

    dfcset = em.read_csv_metadata('cand_set_with_index.csv', key='id', ltable=dfl, 
                                  rtable=dfr, fk_ltable='A_id', fk_rtable='B_id')

    # running debug blocker to identify the records in A x B \ C
    debug_file = em.debug_blocker(dfcset, dfl, dfr)
    
    return debug_file

In [0]:
debug_file = run_debug_blocker(table_a, table_b, '_id', '_id', candidate_set)

No handlers could be found for logger "py_entitymatching.io.parsers"


In [0]:
debug_file

,_id,ltable__id,rtable__id,ltable_title,ltable_release_year,ltable_movie_rating,ltable_summary,ltable_Up_System,rtable_title,rtable_release_year,rtable_movie_rating,rtable_summary,rtable_Up_System
0,0,687,1521,The Exorcist III,1990,6.3,A police Lieutenant uncovers more than he bargained for as his investigation of a series of murd...,CM,The Keys to the House,2004,7.4,"The story of a young father forging a relationship with his handicapped son, whom he meets for t...",CM
1,1,1879,383,Tom and Huck,1995,5.6,Two best friends witness a murder and embark on a series of adventures in order to prove the inn...,CM,Song of the Sea,2014,8.3,"Ben and his little sister Saoirse, the last Seal-child, embark on a fantastic journey across a f...",CM
2,2,1348,991,Zoolander,2001,6.6,"At the end of his career, a clueless fashion model is brainwashed to kill the Prime Minister of ...",CM,Harry Is Here to Help,2001,6.6,A spiraling descent into fear and the perils of relationships in the tradition of Hitchcock. (Mi...,CM
3,3,1998,1669,Empire of the Sun,1987,7.8,A young English boy struggles to survive under Japanese occupation during World War II.,CM,The Great Escape,1963,9.1,Allied prisoners of war plan for several hundred of their number to escape from a German camp du...,CM
4,4,3150,1224,Syriana,2005,6.9,A politically charged epic about the state of the oil industry in the hands of those personally ...,CM,All About Lily Chou-Chou,2002,7.5,An epic story of a Japanese teenager's life in the age of the Internet.,CM
5,5,2547,2909,The Lord of the Rings: The Two Towers,2002,8.7,"While Frodo and Sam edge closer to Mordor with the help of the shifty Gollum, the divided fellow...",CM,The Lord of the Rings: The Fellowship of the Ring,2001,9.1,"An epic adventure of good against evil, a story of the power of friendship and individual courag...",CM
6,6,1235,1558,Super 8,2011,7.0,"During the summer of 1979, a group of friends witness a train crash and investigate subsequent u...",CM,7 Days in September,2002,7.4,Chronicles the life and events of 28 New Yorkers during the week of 9/11.,CM
7,7,666,736,Boogie Nights,1997,7.9,The story of a young man's adventures in the California pornography industry of the late 1970s a...,CM,Raja,2004,tbd,"Shot entirely in Morocco and Marrakech, Raja is the story of a beautiful young Moroccan woman li...",CM
8,8,2397,2614,Man on the Moon,1999,7.4,The life and career of legendary comedian Andy Kaufman.,CM,Charlie: The Life and Art of Charles Chaplin,2004,8.0,The title says it all.,CM
9,9,1875,2983,The Flamingo Kid,1984,6.2,"A recent high school graduate unsure of his future gets a summer job at the Flamingo beach club,...",CM,The Circus,1929,tbd,The Tramp finds work and the girl of his dreams at a circus.,CM


# Module: estimate_precision_recall
# Description: the below code helps you get an estimation of P/R on the candidate set.

In [0]:
import pandas as pd
from scipy.stats import norm
from numpy import sqrt

delta = .05
Z = norm.ppf(1 - (delta / 2))

def estimate_PR(labeled_pairs, reduced_cands, predicted_matches):
    '''
    labeled_pairs - a pandas dataframe with schema id1,id2,label
                    Note label needs to be Boolean

    reduced_cands - a pandas dataframe with schema id1,id2
    predicted_matches - a pandas dataframe with schema id1,id2
    
    return:
        ( (recall lower bound, recall upper bound), (precision lower bound, precision upper bound) )
    '''

    labeled_pairs.drop_duplicates(inplace=True)
    labeled_pairs.columns = ['id1', 'id2', 'label']
    reduced_cands.columns = ['id1', 'id2']
    reduced_cand_set = set(zip(reduced_cands.id1, reduced_cands.id2))
    predicted_matches = set(zip(predicted_matches.id1, predicted_matches.id2))
    
    # estimate the recall
    # number of positives in the labeled sample
    actual_pos = float(labeled_pairs.label.sum())
    # the maximum number of postives in the candidate set
    max_actual_pos = float(actual_pos + len(reduced_cand_set) - len(labeled_pairs))
    
    # true positives in the labeled sample
    true_pos = float(labeled_pairs.apply(lambda x : (x['id1'], x['id2']) in predicted_matches and x['label'], axis=1).sum())
    #estimated recall
    recall = float(true_pos / actual_pos)

    recall_error = Z * sqrt( ((recall * (1 - recall)) / (actual_pos)) * ((max_actual_pos - actual_pos) / (max_actual_pos - 1)) )


    # estimate Precision
    labeled_set  = set(zip(labeled_pairs.id1, labeled_pairs.id2))
    predicted_pos = float(len(labeled_set & predicted_matches))
    
    predicted_pos_in_reduced_cand_set = float(len(reduced_cand_set & predicted_matches))
    
    alpha =  predicted_pos_in_reduced_cand_set / len(predicted_matches)
    precision = alpha * (true_pos / predicted_pos)
    
    precision_error = alpha * Z * sqrt( ((precision * (1 - precision)) / predicted_pos) * (float((len(predicted_matches) - predicted_pos)) / (len(predicted_matches)  - 1)) )

    return ((recall - recall_error, recall + recall_error),
            (precision - precision_error, precision + precision_error))

# [NEED MODIFICATION: Modify this cell to point to the file location]

In [15]:
# read the labeled pairs file, i.e. the file with the labels
labeled_pairs = pd.read_csv('labeled.csv')[:400]
print(estimate_PR(labeled_pairs, dfc, dfp))

((1.0, 1.0), (0.9722216822159181, 0.9869619912534696))


# Alternatively if you run into issues running this script on your laptop you can use
# https://colab.research.google.com/notebooks/welcome.ipynb